<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_06_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 06 NLP

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path_otzv = "/content/gdrive/My Drive/Colab Notebooks/NLP/отзывы за лето.xls"
data = pd.read_excel(path_otzv)
data.tail(3)

,Rating,Content,Date
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [6]:
from datetime import datetime

# data["Year"] = [datetime.strptime(dt, '%Y-%m-%d').year for dt in data["Date"].values]
data["Month"] = [datetime.strptime(dt, '%Y-%m-%d').month for dt in data["Date"].values]
data["Day"] = [datetime.strptime(dt, '%Y-%m-%d').day for dt in data["Date"].values]
data.head()

,Rating,Content,Date,Month,Day
0,5,It just works!,2017-08-14,8,14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,8,14
2,5,Отлично все,2017-08-14,8,14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,8,14
4,5,"Очень удобно, работает быстро.",2017-08-14,8,14


## Задание

Провести сравнение RNN, LSTM, GRU на датасете отзывов

In [7]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=f65c8124395683d94a2d77744b4028824c8e34e1d2d39a436531222e5c1814e1
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [8]:
pip install pymorphy2

     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 8.2MB 15.5MB/s 


In [9]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [11]:
data['Content_prep'] = data['Content'].apply(preprocess_text)
data.tail()

,Rating,Content,Date,Month,Day,Content_prep
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,6,1,шляпас роот право бесполезный прогаразрабыв ох...
20655,5,Ок,2017-06-01,6,1,около
20656,4,Доволен,2017-06-01,6,1,довольный
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,6,1,песопаснастя рута нужын
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01,6,1,сбербанк бомбовый компания сегодняшний лидер б...


In [144]:
train_corpus = " ".join(data['Content_prep'])

In [145]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens_tot = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [146]:
tokens_filtered = [word for word in tokens_tot if word.isalnum()]

In [147]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

In [148]:
vocabulary = {v: i+100  for i, v in enumerate(dist) if dist[v]>1}

In [149]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return result[-maxlen:] + padding 

In [166]:
data_embeded = np.asarray([text_to_sequence(text, 8) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack(((data['Month'].values+50).reshape(data['Month'].values.shape[0], 1), data_embeded))
data_embeded = np.hstack(((data['Day'].values).reshape(data['Day'].values.shape[0], 1), data_embeded))
data_embeded

array([[  14,   58,  100, ...,    0,    0,    0],
       [  14,   58,  107, ...,  113,  114,  115],
       [  14,   58,  116, ...,    0,    0,    0],
       ...,
       [   1,   56,  274, ...,    0,    0,    0],
       [   1,   56,  525, ...,    0,    0,    0],
       [   1,   56,  182, ...,  427, 3814,    0]])

In [167]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [168]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 10, 1)

In [169]:
def to_one_hot(Y):
    n_col = np.amax(Y)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [170]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

## Сравнение RNN, LSTM и GRU

In [171]:
import tensorflow as tf

# !pip install -q tensorflow-hub
# !pip install -q tensorflow-datasets
import tensorflow_hub as hub
# import tensorflow_datasets as tfds

import keras

from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SimpleRNN, GRU
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from typing import List, Optional, Dict, Tuple

max_sequence_len = 10

In [156]:
def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [25]:
def net_gen(lay_name='GRU', func_act="tanh", units=10):
    model = Sequential()

    model.add(Embedding(np.max(data_embeded)+1, units, input_length=max_sequence_len))
    #model.add(Bidirectional(LSTM(50, return_sequences = True)))
    #model.add(Dropout(0.2))
    model.add(eval(lay_name)(units, activation=func_act))    # model.add(lay_name(units, activation=func_act))
    model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(5, activation='softmax'))
 
    # компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC()])

    return model

In [26]:
def net_iter(model, ep=10, batch_size=512):
    history = model.fit(tr_data,
                        train_y_o,
                        epochs=ep, 
                        validation_data=(vd_data, valid_y_o),
                        verbose=0)
    return history.history

In [27]:
summary_data = pd.DataFrame(data=[[0, 0, 0, 0, 0, 0, 0, 0]], 
                            columns=['layers', 'func_act', 'unit_number', 'mean_train_accuracy', 'mean_val_accuracy', 'variance', 'l_b', 'r_b'])
cnt = 0
lay_name = ['SimpleRNN', 'LSTM', 'GRU']
f_act = ['tanh', 'sigmoid']    # 'relu', 
unit_number = [32, 64]    # , 50
epochs = 15

for l_n in lay_name:
    for f_a in f_act:
        for u_n in unit_number:
            err_score = []
            err_val_score = []
            print(f"Модель: слой {l_n}, функция активации {f_a}, число блоков {u_n}, эпох {epochs}\n")
            model = net_gen(lay_name=l_n, func_act=f_a, units=u_n)
            model.summary()
            for i in range(5):
                print(f"\tИтерация: {i+1}")
                hist = net_iter(model, ep=epochs)
                err_score.append(hist[list(hist.keys())[1]][-1])
                err_val_score.append(hist[list(hist.keys())[3]][-1])

            vr = np.var(err_val_score)
            l_b, r_b = calculate_confidence_interval(err_val_score)
            mean_err_score, mean_err_val_score = np.mean(err_score), np.mean(err_val_score)
            print(f"Ошибки на train. Средняя: {mean_err_score} список: {err_score}")
            print(f"Ошибки на valid. Средняя: {mean_err_val_score} список: {err_val_score}")
            print(f"Дисперсия ошибки на valid: {vr}")
            print(f"Доверительный интервал: {l_b} - {r_b}\n\n")
            summary_data.loc[cnt, ['layers', 'func_act', 'unit_number', 'mean_train_score', 
                                    'mean_val_score', 'variance', 'l_b', 'r_b']] = [l_n, f_a, u_n, 
                                            np.mean(err_score), np.mean(err_val_score), vr, l_b, r_b]
            cnt += 1

Модель: слой SimpleRNN, функция активации tanh, число блоков 32, эпох 15

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 32)            413600    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 416,901
Trainable params: 416,901
Non-trainable params: 0
_________________________________________________________________
	Итерация: 1
	Итерация: 2
	Итерация: 3
	Итерация: 4
	Итерация: 5
Ошибки на train. Средняя: 0.9024798274040222 список: [0.8820685744285583, 0.8837203979492188, 0.9104104638099

In [30]:
summary_data.drop(columns=["mean_train_accuracy", "mean_val_accuracy"])

,layers,func_act,unit_number,variance,l_b,r_b,mean_train_score,mean_val_score
0,SimpleRNN,tanh,32.0,0.000017,0.865822,0.876553,0.902480,0.870917
1,SimpleRNN,tanh,64.0,0.000516,0.816312,0.866609,0.842756,0.834633
2,SimpleRNN,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
3,SimpleRNN,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
4,LSTM,tanh,32.0,0.000007,0.868809,0.875856,0.908089,0.871259
5,LSTM,tanh,64.0,0.000006,0.866739,0.873474,0.909013,0.869705
6,LSTM,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
7,LSTM,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
8,GRU,tanh,32.0,0.000019,0.864886,0.875691,0.904983,0.869554
9,GRU,tanh,64.0,0.000028,0.862530,0.877291,0.908321,0.868601


In [31]:
summary_data.drop(columns=["mean_train_accuracy", "mean_val_accuracy"]).sort_values('mean_val_score', ascending=False)

,layers,func_act,unit_number,variance,l_b,r_b,mean_train_score,mean_val_score
4,LSTM,tanh,32.0,0.000007,0.868809,0.875856,0.908089,0.871259
0,SimpleRNN,tanh,32.0,0.000017,0.865822,0.876553,0.902480,0.870917
5,LSTM,tanh,64.0,0.000006,0.866739,0.873474,0.909013,0.869705
8,GRU,tanh,32.0,0.000019,0.864886,0.875691,0.904983,0.869554
9,GRU,tanh,64.0,0.000028,0.862530,0.877291,0.908321,0.868601
1,SimpleRNN,tanh,64.0,0.000516,0.816312,0.866609,0.842756,0.834633
2,SimpleRNN,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
3,SimpleRNN,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312
6,LSTM,sigmoid,32.0,0.000000,0.816312,0.816312,0.816260,0.816312
7,LSTM,sigmoid,64.0,0.000000,0.816312,0.816312,0.816260,0.816312


## Пробуем лучший вариант

In [172]:
model = Sequential()

model.add(Embedding(np.max(data_embeded)+1, 32, input_length=max_sequence_len))
# model.add(Bidirectional(LSTM(32, return_sequences = False)))
model.add(LSTM(32, activation='tanh', input_shape=(tr_data.shape[1], 1))) # model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [173]:
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 10, 32)            413600    
_________________________________________________________________
lstm_21 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_55 (Dense)             (None, 5)                 165       
Total params: 423,141
Trainable params: 423,141
Non-trainable params: 0
_________________________________________________________________


In [174]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [175]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback], 
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 1s 26ms/step - loss: 1.7360 - auc_27: 0.7888 - val_loss: 1.5831 - val_auc_27: 0.8186
Epoch 2/1000
31/31 [==============================] - 0s 12ms/step - loss: 1.4751 - auc_27: 0.8160 - val_loss: 1.4058 - val_auc_27: 0.8025
Epoch 3/1000
31/31 [==============================] - 0s 10ms/step - loss: 1.3761 - auc_27: 0.8047 - val_loss: 1.3463 - val_auc_27: 0.7989
Epoch 4/1000
31/31 [==============================] - 0s 10ms/step - loss: 1.3263 - auc_27: 0.8006 - val_loss: 1.3053 - val_auc_27: 0.8036
Epoch 5/1000
31/31 [==============================] - 0s 10ms/step - loss: 1.2909 - auc_27: 0.8110 - val_loss: 1.2757 - val_auc_27: 0.8274
Epoch 6/1000
31/31 [==============================] - 0s 10ms/step - loss: 1.2647 - auc_27: 0.8322 - val_loss: 1.2528 - val_auc_27: 0.8572
Epoch 7/1000
31/31 [==============================] - 0s 10ms/step - loss: 1.2439 - auc_27: 0.8742 - val_loss: 1.2360 - val_auc_27: 0.8803
Epoch 8/1000
31/31 [=======

## Пробуем увеличить длину последовательности

In [113]:
data_embeded = np.asarray([text_to_sequence(text, 18) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack(((data['Month'].values+50).reshape(data['Month'].values.shape[0], 1), data_embeded))
data_embeded = np.hstack(((data['Day'].values).reshape(data['Day'].values.shape[0], 1), data_embeded))
data_embeded

array([[ 14,  58, 100, ...,   0,   0,   0],
       [ 14,  58, 103, ...,   0,   0,   0],
       [ 14,  58, 116, ...,   0,   0,   0],
       ...,
       [  1,  56, 274, ...,   0,   0,   0],
       [  1,  56, 525, ...,   0,   0,   0],
       [  1,  56, 182, ...,   0,   0,   0]])

In [114]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [115]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 20, 1)

In [116]:
model = Sequential()

model.add(Embedding(np.max(data_embeded)+1, 32, input_length=20))
# model.add(Bidirectional(LSTM(32, return_sequences = False)))
model.add(LSTM(32, activation='tanh')) # model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(5, activation='softmax'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.AUC()])

In [117]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 20, 32)            418144    
_________________________________________________________________
lstm_16 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_45 (Dense)             (None, 5)                 165       
Total params: 427,685
Trainable params: 427,685
Non-trainable params: 0
_________________________________________________________________


In [118]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [119]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback], 
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 1s 28ms/step - loss: 1.7798 - auc_22: 0.7600 - val_loss: 1.5917 - val_auc_22: 0.8499
Epoch 2/1000
31/31 [==============================] - 0s 11ms/step - loss: 1.4834 - auc_22: 0.8261 - val_loss: 1.4138 - val_auc_22: 0.8411
Epoch 3/1000
31/31 [==============================] - 0s 13ms/step - loss: 1.3835 - auc_22: 0.8244 - val_loss: 1.3525 - val_auc_22: 0.8205
Epoch 4/1000
31/31 [==============================] - 0s 13ms/step - loss: 1.3314 - auc_22: 0.8273 - val_loss: 1.3084 - val_auc_22: 0.8432
Epoch 5/1000
31/31 [==============================] - 0s 12ms/step - loss: 1.2902 - auc_22: 0.8532 - val_loss: 1.2734 - val_auc_22: 0.8640
Epoch 6/1000
31/31 [==============================] - 0s 13ms/step - loss: 1.2606 - auc_22: 0.8739 - val_loss: 1.2475 - val_auc_22: 0.8871
Epoch 7/1000
31/31 [==============================] - 0s 13ms/step - loss: 1.2366 - auc_22: 0.8932 - val_loss: 1.2286 - val_auc_22: 0.8904
Epoch 8/1000
31/31 [=======